In [1]:
import numpy as np
import pandas as pd
import nltk
#nltk.download('punkt') # one time execution
#nltk.download('stopwords')
import re
import os
import codecs
from nltk.tokenize import sent_tokenize

In [2]:
TEXTDIR = "./prepro/text/"
ABSTRACTDIR = "./prepro/abstract/"
CLEANEDDIR= "./prepro/cleaned/"


In [3]:
text_files=os.listdir(TEXTDIR)

In [5]:
curr_file = codecs.open(CLEANEDDIR+'1908.07062v3.Recurrent_Neural_Networks_An_Embedded_Computing_Perspective.txt',"r","utf-8")
curr_file_text = curr_file.read()



In [6]:
sentences = sent_tokenize(curr_file_text)
clean_sentences = []
for s in sentences:
    if len(s) >=10:
        clean_sentences.append(s)

In [7]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [8]:
len(word_embeddings)

400000

In [9]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    sen_new = re.sub("[^a-zA-Z]", " ",sen_new)
    return sen_new

In [10]:
# remove stopwords from the sentences
clean_sentences2 = clean_sentences.copy()
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [11]:
clean_sentences

['introduction recurrent neural networks rnns class neural networks nns dealing applications sequential data inputs outputs ',
 'rnns capture temporal relationship input output sequences introducing feedback feedforward ff neural networks ',
 'thus many applications sequential data speech recognition language translation human activity recognition benefit rnns ',
 'contrast cloud computing edge computing guarantee better response time enhance security running application ',
 'augmenting edge devices rnns grant intelligence process respond sequential problems ',
 'realization embedded platforms edge volume devices imposes optimizations rnn applications ',
 'embedded platforms time constrained systems suffer limited memory power resources ',
 'run rnn applications efficiently embedded platforms rnn applications need overcome restrictions ',
 'a  scope article article study rnn models specifically focus rnn optimizations implementations embedded platforms ',
 'article compares recent impl

In [12]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [13]:
len(sentence_vectors)

1048

In [14]:
# similarity matrix
sim_mat = np.zeros([len(clean_sentences), len(clean_sentences)])
sim_mat.shape

(1048, 1048)

In [15]:
from sklearn.metrics.pairwise import cosine_similarity


In [16]:
for i in range(len(clean_sentences)):
  for j in range(len(clean_sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [17]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [18]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(clean_sentences2)), reverse=True)


In [24]:
# Extract top 10 sentences as the summary
generated_abstract= " "
for i in range(10):
  print(ranked_sentences[i][1])
  generated_abstract=generated_abstract+ranked_sentences[i][1]

the computational complexity of the non linear function changes to  be a single comparison  multiplication and addition  which  may be implemented using shifts and additions.
the transformation from a  regular matrix computation to an irregular application  often results in the use of additional hardware and  computation time to manage data.
one other aspect to  volume                                                                                                                                                     consider is that compression optimization results in decreasing the number of operations in the model before running it.
the extra  non linear layers can also be added in computing the output  from the hidden state vector  this model is called the deep  output rnn model.
however  careful choice of the method used to  transform a dense matrix to a sparse matrix may result  in only a limited impact on accuracy while providing  significant gains in computation time.
thus  we cal

In [25]:
print(generated_abstract)

 the computational complexity of the non linear function changes to  be a single comparison  multiplication and addition  which  may be implemented using shifts and additions.the transformation from a  regular matrix computation to an irregular application  often results in the use of additional hardware and  computation time to manage data.one other aspect to  volume                                                                                                                                                     consider is that compression optimization results in decreasing the number of operations in the model before running it.the extra  non linear layers can also be added in computing the output  from the hidden state vector  this model is called the deep  output rnn model.however  careful choice of the method used to  transform a dense matrix to a sparse matrix may result  in only a limited impact on accuracy while providing  significant gains in computation time.thus  we calcula

In [21]:
curr_file = codecs.open(ABSTRACTDIR+text_files[0],"r","utf-8")
original_abstract = curr_file.read()


In [22]:
from rouge import Rouge 

rouge = Rouge()
scores = rouge.get_scores( original_abstract,new_abstract)

In [23]:
scores

[{'rouge-1': {'f': 0.22505307408387096,
   'p': 0.3333333333333333,
   'r': 0.16987179487179488},
  'rouge-2': {'f': 0.029850741800774447,
   'p': 0.04430379746835443,
   'r': 0.022508038585209004},
  'rouge-l': {'f': 0.21804510795861845,
   'p': 0.26851851851851855,
   'r': 0.18354430379746836}}]